In [2]:
import requests
from requests_futures.sessions import FuturesSession
from requests_futures import sessions
import json
import psycopg2
from psycopg2 import Error
import pandas.io.sql as psql
import pandas as pd
from datetime import datetime, timedelta

In [41]:
headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

try:
    conn = psycopg2.connect(dbname='dfpglck7sd4deq', user='vfpfcrcywbbppp',
                            password='2306588cdf3b2662603049a475863835f2ac89cdd52af890323a181d0405601c',
                            host='ec2-52-0-114-209.compute-1.amazonaws.com', port='5432', sslmode='require')
    cur = conn.cursor()

    print('Connected!')

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

Connected!


In [43]:
def check(params):
    session = sessions.FuturesSession(max_workers=1)
    futures = [session.get(url="https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin",headers=headers, params=p) for p in  params]
    for f in futures:
        f.result()

    session.close()
    
    return futures

In [44]:
table = psql.read_sql('SELECT * FROM home_customer', conn)

pins=table.pin.unique()
avail = [0 for i in range(len(pins))]

current = []
for pin in pins:
    current.append(table[table['pin']==pin]['flag2'].values[0])

params=[]
date = datetime.today()

for pin in pins:
    params.append({'pincode': str(pin), 'date': str(date.strftime("%d-%m-%Y"))})

responses = check(params) 

count=0
for i in range(len(pins)):
    res = responses[i].result().json()


    print(count)
    count+=1

    if list(res.keys())[0]=='sessions':
        if len(res['sessions'])!=0:
            for session in res['sessions']:
                for j in range(len(session)):
                    if session['available_capacity'] > 0 and session['min_age_limit']==18:
                        avail[i] = 1


    elif list(res.keys())[0]=='centers':
        if len(res['centers'])!=0:
            for center in res['centers']:
                for j in range(len(center['sessions'])):
                    if center['sessions'][j]['available_capacity'] > 0 and center['sessions'][j]['min_age_limit']==18:
                        avail[i] = 1

0
1
2
3
4
5
6
7


In [47]:

for i in range(len(pins)):
    if avail[i]!=current[i]:
        if avail[i]>current[i]:
            people = table[table['pin']==pins[i]]

            for j in range(len(people)):
                            requests.post(
                                "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                                auth=("api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                                data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                                    "to": people[j]['email'].values[0],
                                    "subject": "Thank for registering to WhenIsMyVaccine",
                                    "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pin}"})

            cur.execute(f'UPDATE home_customer SET flag1=1, flag2=1 WHERE pin={pins[i]};')
            conn.commit()
            

        elif avail[i]<current[i]:
            cur.execute(f'UPDATE home_customer SET flag1=0, flag2=0 WHERE pin={pins[i]};')
            conn.commit()


        elif avail[i]==1:
            peope = table[table['pin']==pins[i] and (table['flag2']==1 and table['flag1']==0)]

            for person in people:
                requests.post(
                                "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                                auth=("api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                                data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                                    "to": people[j]['email'].values[0],
                                    "subject": "Thank for registering to WhenIsMyVaccine",
                                    "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pin}"})

            cur.execute(f'UPDATE home_customer SET flag1=1 WHERE flag1=0 and pin={pins[i]};')
            conn.commit()


In [52]:
people.append({
    'id' : '2123213',
    'pin':  12234,
    'flag1':0,
    'flag2':0,
    'email':'naman',
    'phone':123
})

,id,pin,flag1,flag2,email,phone
11,1678cec0-3549-41d6-ab8d-aa2cf974c939,302018,1,1,nitya.avi1234@gmail.com,9079795789
